<a href="https://colab.research.google.com/github/alexbrill/tf-train/blob/main/imdb%2Blstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import

In [59]:
import numpy as np
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense, Conv1D, LSTM, MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

# data preprocessing

In [8]:
(training_data, training_targets), (testing_data, testing_targets) = imdb.load_data(num_words=10000)
data = np.concatenate((training_data, testing_data), axis=0)
targets = np.concatenate((training_targets, testing_targets), axis=0)

<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/keras/datasets/imdb.py:155: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.7/dist-packages/keras/datasets/imdb.py:156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the nda

# fitting

In [43]:
max_review_length = 500

X_train = sequence.pad_sequences(training_data, maxlen=max_review_length)
y_train = training_targets
X_test = sequence.pad_sequences(testing_data, maxlen=max_review_length)
y_test = testing_targets

In [51]:
X_train[X_train >= 5000] = 0
X_test[X_test >= 5000] = 0

## LSTM 

In [18]:
top_words = 5000
embedding_vector_length = 32
max_review_length = 500

model = Sequential()
model.add(Embedding(top_words, embedding_vector_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


In [53]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)
scores = model.evaluate(X_test, y_test, verbose=0)

print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/3
391/391 [==============================] - 281s 720ms/step - loss: 0.3075 - accuracy: 0.8749 - val_loss: 0.3640 - val_accuracy: 0.8449
Epoch 2/3
391/391 [==============================] - 282s 721ms/step - loss: 0.2863 - accuracy: 0.8823 - val_loss: 0.4135 - val_accuracy: 0.8126
Epoch 3/3
391/391 [==============================] - 281s 719ms/step - loss: 0.2367 - accuracy: 0.9066 - val_loss: 0.3184 - val_accuracy: 0.8753
Accuracy: 87.53%


## LSTM + CONV

In [62]:
model = Sequential()

model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100))

model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())


Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 500, 32)           3104      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 250, 32)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 101       
Total params: 216,405
Trainable params: 216,405
Non-trainable params: 0
_________________________________________________________________
None


In [63]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)
scores = model.evaluate(X_test, y_test, verbose=0)


Epoch 1/3
391/391 [==============================] - 252s 407ms/step - loss: 0.5553 - accuracy: 0.6734 - val_loss: 0.3321 - val_accuracy: 0.8575
Epoch 2/3
391/391 [==============================] - 159s 407ms/step - loss: 0.2436 - accuracy: 0.9045 - val_loss: 0.3099 - val_accuracy: 0.8794
Epoch 3/3
391/391 [==============================] - 159s 406ms/step - loss: 0.1954 - accuracy: 0.9257 - val_loss: 0.3027 - val_accuracy: 0.8748
Accuracy: 87.48%
